# Consolidating historical urban population density data

In this module I work to use the popular Pandas data science library for Python to ingest, clean, and merge 90 years of decennial historical data. I use modern interpolation techniques to interfill missing data values between decennial counts, and validate with cross references to external databases.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [12]:
#initialize the pandas dataframe that will hold all historical data
df_columns = ['city', 'square_miles', 'population_density']
unclean_df = pd.DataFrame()
year = 1910

#below function iterates over all the files in the directory and appends them to the main pandas DataFrame
for filename in os.listdir('PopulationDensityData'):
    for line in filename:
        print(line)  #right now, I'm only iterating over direct filenames- need to use open() to open that file and iterate over lines to convert to tabular format

t
a
b
1
4
.
t
x
t
t
a
b
1
5
.
t
x
t
t
a
b
1
6
.
t
x
t
t
a
b
1
7
.
t
x
t
t
a
b
1
8
.
t
x
t
t
a
b
1
9
.
t
x
t
t
a
b
2
0
.
t
x
t
t
a
b
2
1
.
t
x
t
t
a
b
2
2
.
t
x
t
